### Pandas Lab -- Cleaning, Merging, & Grouping

This lab is designed to introduce students to common use cases for Pandas when working with data:

 - Creating new information out of your existing data set
 - Merging, concatenating, and joining different data sources
 - Grouping -- With both time & non-time based data

### Section I: Creating Data Out of Your Existing Columns

Go ahead and create the following columns in your dataset.

In [189]:
import pandas as pd
import numpy as np

In [190]:
path = r"C:\Users\iulia\OneDrive\Documents\Data Science\superstore.xlsm"
orders = pd.read_excel(path, sheet_name =0)
returns = pd.read_excel(path,sheet_name=1)
person = pd.read_excel(path,sheet_name=2)

**Column 1:**

  - **Column Name:** Profitable
  - **Values:** `True` if `Profit` > 0, `False` if not.

In [191]:
orders['Profitable'] = np.where(orders['Profit']>0,True,False)

**Column 2:**

 - **Column Name:** Expected Ship Time
 - **Values:**
   - `0` if `Ship Mode` == `Same Day`
   - `2` if `Ship Mode` == `First Class`
   - `3` if `Ship Mode` == `Second Class`
   - `6` if `Ship Mode` == `Standard Class`
   - `-1` if none of the above.

In [192]:
conditions = [
    orders['Ship Mode'] == 'Same Day',
    orders['Ship Mode'] == 'First Class',
    orders['Ship Mode'] == 'Second Class',
    orders['Ship Mode'] == 'Standard Class'
]
results = [0,2,3,6]
orders['Expected Ship Time'] = np.select(conditions, results,-1)

**Column 3:**

 - **Column Name:** Actual Ship Time
 - **Values:**
   - `Ship Date` - `Order Date`
 - **Note:** When you subtract these columns, your column will be a **time delta**.  See if you can use the `dt` attribute to convert these values into an integer.  Ie, if your value reads `3 days`, you want that to be 3 instead.  You can read more about different time periods in pandas here:  https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#time-date-components

In [193]:
#orders['Actual Ship Time']=orders['Ship Date'] - orders['Order Date']
orders['Actual Ship Time'] = (orders['Ship Date'] - orders['Order Date']).dt.days

**Column 4:**

 - **Column Name:** Late
 - **Values:** `True` if `Actual Ship Time` > `Expected Ship Time`, `False` otherwise

In [194]:
orders['Late']=orders['Actual Ship Time']>orders['Expected Ship Time']

### Section II: Merging Dataframes

This excel spreadsheet has 3 separate sheets.  Look up the documentation on the `pd.read_excel` method on how to load in the other two.  

After that, merge the other two dataframes into your original one, and make sure your original dataset now has the following columns:

 - **Salesperson:** This is the Salesperson in charge of each region.
 - **Returned:** This details whether or not the order was returned.  Fill in null values with the value `no`.
 
Use the `drop()` method if you need to get rid of redundant columns.

**Important:** We want to keep all of the rows in the dataset we first loaded in.  After each merge, it's a good idea to make sure your dataset hasn't shrunk, which will happen if you don't choose the right merge type.  Make sure you have 9,994 rows when you're finished!

In [195]:
orders = orders.merge(person, on = 'Region', how = 'left')
orders = orders.merge(returns, on = 'Order ID',how ='left')

In [196]:
orders['Returned'].fillna('No',inplace=True)

In [197]:
orders.rename({'Person':'Salesperson'},axis=1, inplace = True)

### Section III: Grouping

Use the `groupby` or `resample` method to answer the following questions.

**Question 1:** What salesperson had the highest average sales amount? 

In [198]:
orders.groupby('Salesperson')['Sales'].mean()

Salesperson
Anna Andreadi        226.493233
Cassandra Brandow    241.803645
Chuck Magee          238.336110
Kelly Williams       215.772661
Name: Sales, dtype: float64

**Question 2:** Within each ship mode, compare how likely late orders were to be profitable or not

In [199]:
orders.groupby(['Ship Mode','Late'])['Profitable'].mean()

Ship Mode       Late 
First Class     False    0.795405
                True     0.814103
Same Day        False    0.813102
                True     0.916667
Second Class    False    0.845415
                True     0.827500
Standard Class  False    0.794651
                True     0.805153
Name: Profitable, dtype: float64

**Question 3:** What ship mode had the most consistently on time orders?

In [200]:
orders.groupby('Ship Mode')['Late'].mean()

Ship Mode
First Class       0.405722
Same Day          0.044199
Second Class      0.411311
Standard Class    0.104055
Name: Late, dtype: float64

**Question 4:** For each sales person, get their average, median, max, and count of their sales.

In [201]:
orders.groupby('Salesperson')['Sales'].agg([np.mean,np.median,'max','count'])

,mean,median,max,count
Salesperson,,,,
Anna Andreadi,226.493233,60.840,13999.960,3203
Cassandra Brandow,241.803645,54.594,22638.480,1620
Chuck Magee,238.336110,54.900,11199.968,2848
Kelly Williams,215.772661,45.980,17499.950,2323


**Question 5:** Group your dataset according to  `Region`, and `Category`, and then call the `describe()` method to get the summary statistics for each subgroup.

In [202]:
orders.groupby(['Region','Category']).describe()

Row ID                                           \
                          count         mean          std    min     25%   
Region  Category                                                           
Central Furniture         481.0  5198.831601  2965.518718   37.0  2788.0   
        Office Supplies  1422.0  5025.764416  2890.995980   15.0  2601.5   
        Technology        420.0  4854.019048  2980.905419   36.0  2045.0   
East    Furniture         601.0  4976.960067  2927.036861   24.0  2330.0   
        Office Supplies  1712.0  4926.139019  2971.616543   29.0  2194.5   
        Technology        535.0  4951.805607  2892.183235   48.0  2432.5   
South   Furniture         332.0  5019.783133  2848.584072    1.0  2699.5   
        Office Supplies   995.0  5105.296482  2778.947757    5.0  2693.0   
        Technology        293.0  5006.679181  2774.039447  107.0  2780.0   
West    Furniture         707.0  4999.953324  2814.077939    6.0  2614.5   
        Office Supplies  1897.0  4929.139694  2841.198338    3.0  2450.0   
        Technology        599.0  5152.409015  2910.770129    8.0  2720.5   

                                                 Postal Code                \
                            50%      75%     max       count          mean   
Region  Category                                                             
Central Furniture        5168.0  7949.00  9963.0       481.0  65486.453222   
        Office Supplies  4948.0  7584.50  9983.0      1422.0  65275.734880   
        Technology       4793.0  7373.75  9984.0       420.0  65393.026190   
East    Furniture        4826.0  7562.00  9965.0       601.0  17317.247920   
        Office Supplies  4929.0  7513.25  9986.0      1712.0  17401.548481   
        Technology       5040.0  7420.00  9947.0       535.0  18640.338318   
South   Furniture        5322.0  7405.50  9990.0       332.0  34552.117470   
        Office Supplies  5053.0  7519.50  9972.0       995.0  34243.389950   
        Technology       4912.0  7479.00  9989.0       293.0  35261.351536   
West    Furniture        5145.0  7360.00  9991.0       707.0  91680.442716   
        Office Supplies  4895.0  7319.00  9994.0      1897.0  91769.716921   
        Technology       5401.0  7669.50  9992.0       599.0  91543.449082   

                         ... Expected Ship Time      Actual Ship Time  \
                         ...                75%  max            count   
Region  Category         ...                                            
Central Furniture        ...                6.0  6.0            481.0   
        Office Supplies  ...                6.0  6.0           1422.0   
        Technology       ...                6.0  6.0            420.0   
East    Furniture        ...                6.0  6.0            601.0   
        Office Supplies  ...                6.0  6.0           1712.0   
        Technology       ...                6.0  6.0            535.0   
South   Furniture        ...                6.0  6.0            332.0   
        Office Supplies  ...                6.0  6.0            995.0   
        Technology       ...                6.0  6.0            293.0   
West    Furniture        ...                6.0  6.0            707.0   
        Office Supplies  ...                6.0  6.0           1897.0   
        Technology       ...                6.0  6.0            599.0   

                                                                      
                             mean       std  min  25%  50%  75%  max  
Region  Category                                                      
Central Furniture        3.979210  1.713787  0.0  3.0  4.0  5.0  7.0  
        Office Supplies  4.120253  1.707383  0.0  3.0  4.0  5.0  7.0  
        Technology       3.938095  1.586331  0.0  3.0  4.0  5.0  7.0  
East    Furniture        3.850250  1.747628  0.0  3.0  4.0  5.0  7.0  
        Office Supplies  3.937500  1.724834  0.0  3.0  4.0  5.0  7.0  
        Technology       3.882243  1.720977  0.0 

**Question 6:** Use the `Resample()` method to get the sum of sales for each quarter.

In [203]:
orders.resample('Q', on='Order Date')['Sales'].sum()

Order Date
2014-03-31     74447.7960
2014-06-30     86538.7596
2014-09-30    143633.2123
2014-12-31    179627.7302
2015-03-31     68851.7386
2015-06-30     89124.1870
2015-09-30    130259.5752
2015-12-31    182297.0082
2016-03-31     93237.1810
2016-06-30    136082.3010
2016-09-30    143787.3622
2016-12-31    236098.7538
2017-03-31    123144.8602
2017-06-30    133764.3720
2017-09-30    196251.9560
2017-12-31    280054.0670
Freq: Q-DEC, Name: Sales, dtype: float64

**Question 7:** What quarter had the highest total sales amount?

In [204]:
orders.resample('Q',on ='Order Date')['Sales'].sum().idxmax()

Timestamp('2017-12-31 00:00:00', freq='Q-DEC')

**Question 8:** See if you can use the `groupby` method to get a list of yearly sales for each region inside the dataset.

**Hint:** Try using the `dt` attribute of the `Order Date` column.

In [205]:
orders.resample('y',on='Order Date')['Sales'].sum()

Order Date
2014-12-31    484247.4981
2015-12-31    470532.5090
2016-12-31    609205.5980
2017-12-31    733215.2552
Freq: A-DEC, Name: Sales, dtype: float64

In [206]:
orders.groupby(orders['Order Date'].dt.year)['Sales'].sum()

Order Date
2014    484247.4981
2015    470532.5090
2016    609205.5980
2017    733215.2552
Name: Sales, dtype: float64

**Bonus:** Creating summary statistics with a `groupby` statement.  

Lots of times it's very useful to be able to create a summary statistic for a particular category to compare with individual samples.

For example, if you were doing fraud detection, and someone were making a purchase at a 7-11, a transaction amount of $175 would be unusually large for someone making a purchase at a convenience store, essentially setting off a red flag that the transaction might be suspicious.

Creating such comparisons is easily done using the `groupby` method and then merging it back into the original dataframe.  

For example, if you wanted to compare every single purchase amount with the average amount for that category, you could do it in the following way:

In [211]:
import pandas as pd
df = pd.read_excel(r'C:\Users\iulia\OneDrive\Documents\Data Science\superstore.xlsm')
# create the grouping
cat_grouping = df.groupby('Category')[['Sales']].mean()
# this step is mostly just to make the merged dataframe more tidy
cat_grouping.rename({'Sales': 'Cat_Average'}, axis=1, inplace=True)

In [212]:
# join them
df = df.merge(cat_grouping, left_on='Category', right_index=True)

In [209]:
# and now we can see each purchase amount compared to the average amt
# for that category
df.head()

NameError: name 'df' is not defined

So for instance, if wanted to ask ourselves, "which customers consistently punch above their weight when it comes to the actual items that they buy?"

We could easily do the following:

In [210]:
# turn the difference between the two columns into a percent
df['Cat Difference'] = ((df['Sales'] / df['Cat_Average']) - 1) * 100

NameError: name 'df' is not defined

In [6]:
# now group and sort the values
df.groupby('Customer Name')['Cat Difference'].mean().sort_values(ascending=False)

Customer Name
Mitch Willingham        845.569285
Christopher Martinez    636.313301
Andy Reiter             449.654450
Adrian Barton           418.548520
Sanjit Chand            386.906876
Amy Cox                 363.031889
Yoseph Carroll          344.794116
Yana Sorensen           317.572571
Sean Miller             308.018602
Tamara Chand            304.928500
Alex Avila              296.561353
Greg Maxwell            292.917810
Grant Thornton          291.980410
Jane Waco               272.432920
Tom Ashbrook            270.172385
Paul Knutson            244.824972
Robert Dilbeck          239.904239
Gary Hwang              231.081468
Ken Lonsdale            216.273514
Dennis Pardue           213.919098
Justin Hirsh            205.806859
Stefanie Holloman       199.625920
Cathy Prescott          194.089483
Bill Shonely            194.068945
Erica Smith             193.504802
Neil Ducich             174.357173
Adam Bellavance         174.177216
Hunter Lopez            170.364451
Karen 

**Your Turn:** Using a similar methodology as above, figure out the 10 customers who are the most profitable on average, when compared to the subcategory that they purchased from.

If you wanted, you could also limit this to customers who only made a minimum number of purchases as well.

In [ ]:
# your answer here